In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
CONFIG_RAN_ALL_THE_WAY = False

###############################################################################
# Imports
###############################################################################
# Standard library
import logging
from pathlib import Path
import sys

from typing import *

# Third party
import colorama
import transformers
assert int(transformers.__version__.split('.')[0]) >= 4, transformers.__version__

# First Party
import iterated_utils as utils
import iterated_retrieval as ir
import iterated_retrieval
import common_retriever


ROOT_PATH = Path("/home/mila/g/gagnonju/IteratedDecoding/")

assert "condaless" in sys.executable, sys.executable


###############################################################################
# Logging
###############################################################################
LOGGER = logging.getLogger(__name__)

format_info = (
    "[%(levelname)s] (%(asctime)s) "
    "{%(name)s.%(funcName)s:%(lineno)d}:\n"
)

logging_format = (
    colorama.Fore.CYAN +
    format_info +
    colorama.Style.RESET_ALL +
    "%(message)s"
)
logging.basicConfig(
    format=logging_format,
    level=logging.WARNING,
    force=True,
)
logging.getLogger(
    "transformers.configuration_utils"
).setLevel(logging.WARN)
logging.getLogger(
    "transformers.tokenization_utils"
).setLevel(logging.WARN)
logging.getLogger(
    "transformers.modeling_utils"
).setLevel(logging.WARN)
logging.getLogger(
    "common_retriever"
).setLevel(logging.INFO)
logging.getLogger(
    "dense_retriever"
).setLevel(logging.INFO)



###############################################################################
# CONFIG
###############################################################################
conf_path = ROOT_PATH / "jobs" / "retrieve_and_decode" / "config.json"
args, dpr_cfg = ir.build_args(config_path=conf_path, root_path=ROOT_PATH)

(
    dataloader, tokenizer_bart, tokenizer_bert,
) = iterated_retrieval.build_tokenizers_and_datasets(
    generation_batch_size=args.generation_batch_size,
    data_dir=args.data_dir,
    max_target_len=args.dataloader_max_target_len,
    max_source_len=args.dataloader_max_source_len,
    cv_set=args.cv_set
)


LOGGER.info("Done.")
CONFIG_RAN_ALL_THE_WAY = True

data_dir = PosixPath('/home/mila/g/gagnonju/IteratedDecoding/GAR/data/nq-answer')

loading from /home/mila/g/gagnonju/IteratedDecoding/GAR/data/nq-answer/val.target.processed (pkl)... make sure data is what you need


In [5]:
retriever, all_passages, special_query_token = common_retriever.build_retriever(
    dpr_cfg,
    ROOT_PATH / "jobs" / "retrieve_and_decode" / "cache" 
)
retriever.index.index = common_retriever.faiss_to_gpu(
    retriever.index.index,
)


[INFO] (2021-11-15 10:16:44,852) {common_retriever.make_retriever:217}:
Loading encoders.


(Re)/Loading hf_models.py
hf_models.py: Third Party
4.12.0
hf_models.py: First Party
hf_models.py:  Code
hf_model.py: Done.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing HFBertEncoder: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing HFBertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HFBertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing HFBertEncoder: ['cls.predictions.transform.dense.bias

In [ ]:
query_aug_model, reader_model = ir.build_models(
    reader_model_path=args.reader_model_path,
    query_aug_model_path=args.query_aug_model_path,
)

In [21]:
assert CONFIG_RAN_ALL_THE_WAY

ir.inference(
    all_passages=all_passages,
    query_aug_model=query_aug_model.cuda(),
    reader_model=reader_model.cuda() if reader_model else None,
    special_query_token=special_query_token,
    retriever=retriever,
    selection_technique=ir.selection_technique,
    question_dataloader=dataloader,
    max_loop_n=args.max_loop_n,
    query_aug_input_max_length=args.max_source_len,
    decoding_conf_query_aug=args.decoding_conf_query_aug,
    decoding_conf_reader=args.decoding_conf_reader,
    n_docs=args.n_docs,
    out_path=args.out_path,
    retriever_batch_size=args.retriever_batch_size,
    aug_method=args.aug_method,
    final_num_contexts=args.final_num_contexts,
    generation_batch_size=args.generation_batch_size,
    selection_mode=args.selection_mode,
    tokenizer_bart=tokenizer_bart,
    tokenizer_bert=tokenizer_bert,
    augmentation_mode=args.augmentation_mode,
)

(Re)/Loading iterated_retrieval.py


{
    'num_return_sequences': 5,
    'max_length': 40,
    'num_beams': None,
    'num_beam_groups': None,
    'temperature': 5.0,
    'early_stopping': True,
    'do_sample': True
}

[loop_i = 0] Preparing the retrieval queries ::  question_generator: 100%|██████████| 1752/1752 [00:00<00:00, 4028.69it/s]


/home/mila/g/gagnonju/IteratedDecoding/jobs/retrieve_and_decode/iterated_decoding_output/y2021m11d15-h10m21s57_ANSWER_CONCATENATE_ARGMAX_W_AUG_100_FINAL_NUM_CONTEXTS_100_TEMPERATURE_5.0/retr_inputs_0.jsonl


retrieval all_queries_this_loop:  57%|█████▋    | 5020/8757 [03:22<02:29, 24.92it/s]